In [3]:
import os
import pandas as pd

os.chdir(r"C:\Users\zirui\OneDrive\Desktop\238\CS238_Project")

input_path = r"data/weather_data/producing_locations/hurdat2.csv"
output_path = r"data/weather_data/producing_locations/hurdat2_processed.csv"

df = pd.read_csv(input_path)

df["date"] = pd.to_datetime(df["date"], format="%Y%m%d")
df = df[df["date"].dt.year >= 1986]

tier1_cols = [
    "date", "time", "status_of_system",
    "latitude", "longitude",
    "maximum_sustained_wind_knots", "central_pressure_mb"
]

df[tier1_cols].to_csv(output_path, index=False)


In [4]:
import pandas as pd

df = pd.read_csv(r"data/weather_data/producing_locations/hurdat2_processed.csv")

print(df["status_of_system"].nunique())
print(df["status_of_system"].value_counts())


9
status_of_system
TS    6228
HU    4195
TD    3418
EX    2212
LO    1638
SS     319
DB     181
SD     128
WV     127
Name: count, dtype: int64


In [6]:
import numpy as np

df = pd.read_csv(r"data/weather_data/producing_locations/hurdat2_processed.csv")

def parse_lat(s):
    return float(s[:-1]) * (1 if s.endswith("N") else -1)

def parse_lon(s):
    return float(s[:-1]) * (-1 if s.endswith("W") else 1)

df["lat"] = df["latitude"].apply(parse_lat)
df["lon"] = df["longitude"].apply(parse_lon)

df["lat_rad"] = np.deg2rad(df["lat"])
df["lon_rad"] = np.deg2rad(df["lon"])

df["x"] = np.cos(df["lat_rad"]) * np.cos(df["lon_rad"])
df["y"] = np.cos(df["lat_rad"]) * np.sin(df["lon_rad"])
df["z"] = np.sin(df["lat_rad"])

df = df.drop(columns=["latitude", "longitude", "time", "lat_rad", "lon_rad"])

dummies = pd.get_dummies(df["status_of_system"])
df = pd.concat([df.drop(columns=["status_of_system"]), dummies], axis=1)

df = df.groupby("date", as_index=False).mean(numeric_only=True)

df.to_csv(r"data/weather_data/producing_locations/hurdat2_processed_cleaned.csv", index=False)
